In [1]:
import dace
import typing
import os
import numpy as np
from dace.transformation.dataflow import DoubleBuffering, MapTiling
from dace.transformation.soft_hier import CannonTransformer, SplitHBMLoad, CanonTransformer

def make_preload_elf(output_file_path, np_arrays, start_addresses=None):
    """
    Generate an ELF file preloading numpy arrays.

    Parameters:
    - output_file_path (str): Path to save the output ELF file.
    - np_arrays (list of numpy.ndarray): List of numpy arrays to include in the ELF.
    - start_addresses (list of int or None): List of starting addresses for each array, or None.
      If None, addresses are auto-determined with 64-byte alignment.
    """
    NP_DTYPE_TO_C = {
        np.dtype('int8'): 'int8_t',
        np.dtype('uint8'): 'uint8_t',
        np.dtype('int16'): 'int16_t',
        np.dtype('uint16'): 'uint16_t',
        np.dtype('int32'): 'int32_t',
        np.dtype('uint32'): 'uint32_t',
        np.dtype('int64'): 'int64_t',
        np.dtype('uint64'): 'uint64_t',
        np.dtype('float16'): 'float16',
        np.dtype('float32'): 'float',
        np.dtype('float64'): 'double',
    }
    
    ENV_PATH = os.environ.get("PATH")
    # Add RISC-V toolchain to PATH /scratch/dace4softhier/gvsoc/third_party/toolchain/v1.0.16-pulp-riscv-gcc-centos-7/bin/
    os.environ["PATH"] = f"{ENV_PATH}:/usr/scratch/badile111/dace4softhier/gvsoc/third_party/toolchain/v1.0.16-pulp-riscv-gcc-centos-7/bin/"
    
    # Handle default for start_addresses
    if start_addresses is None:
        start_addresses = [None] * len(np_arrays)

    # Validate inputs
    if len(np_arrays) != len(start_addresses):
        raise ValueError("np_arrays and start_addresses must have the same length.")

    # 64-byte alignment
    alignment = 64
    current_address = 0xc0000000  # Default starting address for auto-addressing

    # Step 1: Create "array.c"
    array_c_content = ['#include <stdint.h>']
    section_names = []

    for idx, (array, start_addr) in enumerate(zip(np_arrays, start_addresses)):
        # Determine C type from NumPy dtype
        c_type = NP_DTYPE_TO_C.get(array.dtype, None)
        if c_type is None:
            raise TypeError(f"Unsupported NumPy dtype: {array.dtype}")

        section_name = f".custom_section_{idx}"
        section_names.append(section_name)
        
        if start_addr is None:
            # Auto-determine the address with alignment
            start_addr = (current_address + alignment - 1) & ~(alignment - 1)
        else:
            # Ensure provided addresses are aligned
            if start_addr % alignment != 0:
                raise ValueError(f"Provided address {start_addr} is not {alignment}-byte aligned.")

        # Generate the array definition
        array_values = ", ".join(map(str, array.flatten()))
        array_c_content.append(
            f'{c_type} array_{idx}[] __attribute__((section("{section_name}"))) = {{{array_values}}};'
        )

        current_address = start_addr + array.nbytes

    array_c_code = "\n".join(array_c_content)

    with open("array.c", "w") as f:
        f.write(array_c_code)


    # Step 2: Create "link.ld"
    link_ld_content = ["SECTIONS {"]
    current_address = 0xc0000000  # Reset for linker script auto-addressing

    for idx, (array, start_addr) in enumerate(zip(np_arrays, start_addresses)):
        section_name = section_names[idx]

        if start_addr is None:
            # Auto-determine the address with alignment
            start_addr = (current_address + alignment - 1) & ~(alignment - 1)
        link_ld_content.append(
            f"    . = 0x{start_addr:X};\n    {section_name} : {{ *({section_name}) }}"
        )
        current_address = start_addr + array.nbytes

    link_ld_content.append("}")
    link_ld_code = "\n".join(link_ld_content)

    with open("link.ld", "w") as f:
        f.write(link_ld_code)

    # Step 3: Compile the ELF file
    os.system("riscv32-unknown-elf-gcc -c array.c -o array.o")
    os.system(f"riscv32-unknown-elf-ld -T link.ld array.o -o {output_file_path}")
    os.system(f"riscv32-unknown-elf-strip --remove-section=.comment --remove-section=.Pulp_Chip.Info {output_file_path}")

    # Step 4: Cleanup
    os.remove("array.c")
    os.remove("link.ld")
    os.remove("array.o")


def make_preload_elf_hbm_interleaved(output_file_path, np_arrays, split_schemes, placement_schemes, hardware_block_sizes, start_addresses=None, KMN=None, args_only=True):
    """
    Split np arrays into tiles and blocks and then use make_preload_elf to generate an ELF file preloading numpy arrays.

    """
     # 1) Combine relevant info with original indices
    arrays_info = [
        (idx, array, split_schemes[idx], placement_schemes[idx], hardware_block_sizes[idx])
        for idx, array in enumerate(np_arrays)
    ]

    # Helper to compute the number of channels from a placement scheme
    def get_num_channels(placement_scheme):
        start_channel, end_channel, stride = placement_scheme
        return (end_channel - start_channel + 1) // stride

    # 2) Sort by number of channels used, descending
    arrays_info.sort(key=lambda x: get_num_channels(x[3]), reverse=True)
    #        index: x[0]
    #        array: x[1]
    # split_scheme: x[2]
    #placement_scheme: x[3]
    #hardware_block_size: x[4]

    # 3) Prepare channel start addresses
    total_channels = 8
    current_start_address = 64
    start_addr_in_each_channel = [current_start_address] * total_channels

    # We'll store each array's start address in a list keyed by original index
    # so we can return them in the original order.
    start_addresses_in_original_order = [None] * len(np_arrays)

    # 4) Allocate arrays in sorted order
    for idx, array, split_scheme, placement_scheme, hardware_block_size in arrays_info:
        start_channel, end_channel, stride = placement_scheme
        num_channels = get_num_channels(placement_scheme)

        # Basic tile check
        array_shape = array.shape
        tile_height = array_shape[0] // split_scheme[0]
        tile_length = array_shape[1] // split_scheme[1]
        if tile_height < hardware_block_size[0] or tile_length < hardware_block_size[1]:
            raise ValueError(
                f"Invalid tile size: {tile_height}x{tile_length}"
                f" < {hardware_block_size[0]}x{hardware_block_size[1]}"
            )

        # Compute how many bytes total and how many bytes per channel
        array_size = array.nbytes
        array_size_per_channel = array_size // num_channels

        # Grab the start address for the "first" channel in the group
        array_start_addr_in_channel = start_addr_in_each_channel[start_channel]

        # 5) Store that start address in the array's original position
        start_addresses_in_original_order[idx] = array_start_addr_in_channel

        # 6) Advance addresses in all channels used by this array
        for i in range(num_channels):
            channel_idx = start_channel + i * stride

            # Optional: Validate all channels in the group are at the same start
            if start_addr_in_each_channel[channel_idx] != array_start_addr_in_channel:
                raise ValueError(
                    f"Invalid placement scheme: {placement_scheme}, "
                    f"channel {channel_idx} start address mismatch"
                )
            start_addr_in_each_channel[channel_idx] += array_size_per_channel

    args = start_addresses_in_original_order
    for arg in args:
        print(f"arg: {hex(arg)}")

      
    split_arrays = []
    split_arrays.append(args)
    split_arrays_start_addresses = []
    split_arrays_start_addresses.append(hbm_node_addr_base) # for store args

    if not args_only:
        for array, split_scheme, placement_scheme, hardware_block_size, arg_start_addr in zip(np_arrays, split_schemes, placement_schemes, hardware_block_sizes, args):
            current_start_address = arg_start_addr
            print(f"current_start_address: {hex(current_start_address)}")
            block_height = hardware_block_size[0]
            block_length = hardware_block_size[1]
            block_size = block_height * block_length * np.dtype(array.dtype).itemsize
            print(f"block_size: {block_size}")
            channel_start = placement_scheme[0]
            print(f"channel_start: {channel_start}")
            channel_end = placement_scheme[1]
            print(f"channel_end: {channel_end}")
            channel_stride = placement_scheme[2]
            print(f"channel_stride: {channel_stride}")
            num_channels = (channel_end - channel_start + 1) // channel_stride
            array_shape = array.shape
            print(f"array_shape: {array_shape}")
            tile_height = array_shape[0] // split_scheme[0]
            print(f"tile_height: {tile_height}")
            tile_length = array_shape[1] // split_scheme[1]
            print(f"tile_length: {tile_length}")
            tile_size = tile_length * tile_height * np.dtype(array.dtype).itemsize
            print(f"tile_size: {tile_size}")
            for i in range(split_scheme[0]):
                for j in range(split_scheme[1]):
                    tile_idx = i * split_scheme[1] + j
                    print(f"tile_idx: {tile_idx}")
                    channel_offset = tile_idx % num_channels
                    print(f"channel_offset: {channel_offset}")
                    channel_idx = channel_start + channel_offset * channel_stride
                    print(f"channel_idx: {channel_idx}")
                    tile = array[i*tile_height:(i+1)*tile_height, j*tile_length:(j+1)*tile_length]
                    for bi in range(0, tile_height, block_height):
                        for bj in range(0, tile_length, block_length):
                            print(f"bi: {bi}, bj: {bj}")
                            block = tile[bi:bi+block_height, bj:bj+block_length] 
                            split_arrays.append(block)
                            bi_index = bi // block_height
                            bj_index = bj // block_length
                            block_address = hbm_node_addr_base + current_start_address + channel_idx * hbm_node_addr_space + (tile_idx // num_channels) * tile_size + (bi_index * tile_length // block_length + bj_index) * block_size
                            print(f"block_address: {hex(block_address)}")
                            split_arrays_start_addresses.append(block_address)

    if KMN is None:
        args.append(K)
        args.append(M)
        args.append(N)
    else:
        args.append(KMN[0])
        args.append(KMN[1])
        args.append(KMN[2])
    # args to np arrays
    args = np.array(args, dtype=np.uint32)  

    # replace the args in split_arrays with new args
    split_arrays[0] = args

    make_preload_elf(output_file_path, split_arrays, split_arrays_start_addresses)

    return args



In [2]:

M = dace.symbol("M")
N = dace.symbol("N")
K = dace.symbol("K")

def _my_gen_matmul_sdfg(hardware_matmul_mnk: typing.Tuple,
                     global_storage: dace.dtypes.StorageType,
                     local_storage: dace.dtypes.StorageType,
                     device_schedule: dace.dtypes.ScheduleType,
                     thread_group_schedule: dace.dtypes.ScheduleType,
                     thread_group_dims: typing.Tuple,
                     hbm_split_scheme: typing.List[typing.Tuple[int, int]],
                     hbm_placement_scheme: typing.List[typing.Tuple[int, int]],
                     input_float,
                     output_float,
                     coarsening_factor,
                     mmad_tasklet_str: str,
                     is_hbm_interleaved: bool = False):
    sdfg = dace.SDFG("GEMM")
    tM, tN, tK = hardware_matmul_mnk
    tM *= coarsening_factor
    tN *= coarsening_factor
    tK *= coarsening_factor
    gM, gN = thread_group_dims

    main_state = sdfg.add_state("main")
    state = main_state

    arrs = dict()
    for arr_name, shape, ftype in [("A", (M, K), input_float), ("B", (K, N), input_float), ("C", (M, N), output_float)]:
        if arr_name == "A":
            arrn, arr = sdfg.add_array(name=arr_name, shape=shape, dtype=ftype, storage=global_storage, transient=False, is_hbm_interleaved=is_hbm_interleaved, hbm_split_scheme=hbm_split_scheme[0], hbm_placement_scheme=hbm_placement_scheme[0])
            arrs[arrn] = arr
        if arr_name == "B":
            arrn, arr = sdfg.add_array(name=arr_name, shape=shape, dtype=ftype, storage=global_storage, transient=False, is_hbm_interleaved=is_hbm_interleaved, hbm_split_scheme=hbm_split_scheme[1], hbm_placement_scheme=hbm_placement_scheme[1])
            arrs[arrn] = arr
        if arr_name == "C":
            arrn, arr = sdfg.add_array(name=arr_name, shape=shape, dtype=ftype, storage=global_storage, transient=False, is_hbm_interleaved=is_hbm_interleaved, hbm_split_scheme=hbm_split_scheme[2], hbm_placement_scheme=hbm_placement_scheme[2])
            arrs[arrn] = arr
    arrn, arr = sdfg.add_array(name="accumulator", shape=(coarsening_factor*coarsening_factor, tM//coarsening_factor, tN//coarsening_factor), dtype=ftype, storage=local_storage, transient=True)
    arrs[arrn] = arr

    dev_map_entry, dev_map_exit = main_state.add_map(
        name="gemm_entry",
        ndrange={"i" : dace.subsets.Range([(0, M-1, tM*gM)]),
                 "j" : dace.subsets.Range([(0, N-1, tN*gN)])},
        schedule=device_schedule
    )
    for name in ["A", "B", "C"]:
    # for name in ["A", "B"]:
        if name == "A" or name == "B":
            access_str = ", ".join([f"0:{n}" for n in arrs[name].shape])
            an = state.add_access(name)
            state.add_edge(an, None, dev_map_entry, f"IN_{name}", dace.memlet.Memlet(f"{name}[{access_str}]"))
            dev_map_entry.add_in_connector(f"IN_{name}")
        if name == "C":
            access_str = ", ".join([f"0:{n}" for n in arrs[name].shape])
            # an = state.add_access(name)
            dev_map_exit.add_out_connector(f"OUT_{name}")
            anc3 = state.add_access(name)
            state.add_edge(dev_map_exit, f"OUT_{name}", anc3, None, dace.memlet.Memlet(f"{name}[{access_str}]"))

    thread_group_map_entry, thread_group_map_exit = main_state.add_map(
        name="thread_group_mmad",
        ndrange={"gi" : dace.subsets.Range([(0, gM-1, 1)]),
                 "gj" : dace.subsets.Range([(0, gM-1, 1)])},
        schedule=thread_group_schedule
    )

    gi = dace.symbol("gi")
    gj = dace.symbol("gj")
    
    for name in ["A", "B", "C"]:
        if name == "A" or name == "B":
            if name == "A":
                access_str = ", ".join([f"i:i + {tM} * {gM}", "0:K"])
            elif name == "B":
                access_str = ", ".join(["0:K", f"j:j + {tN} * {gN}"])
            elif name == "C":
                access_str = ", ".join([f"i:i + {gM} * {tM}", f"j:j + {gN} * {tN}"])
            state.add_edge(dev_map_entry, f"OUT_{name}", thread_group_map_entry, f"IN_{name}", dace.memlet.Memlet(f"{name}[{access_str}]"))
            dev_map_entry.add_out_connector(f"OUT_{name}")
            thread_group_map_entry.add_in_connector(f"IN_{name}")
        if name == "C":
            access_str = ", ".join([f"i:i + {gM} * {tM}", f"j:j + {gN} * {tN}"])
            state.add_edge(thread_group_map_exit, f"OUT_{name}", dev_map_exit, f"IN_{name}", dace.memlet.Memlet(f"{name}[{access_str}]"))
            dev_map_exit.add_in_connector(f"IN_{name}")
            thread_group_map_exit.add_out_connector(f"OUT_{name}")

    thread_coarsened_map_entry, thread_coarsened_map_exit = main_state.add_map(
        name="thread_coarsened",
        ndrange={"ci" : dace.subsets.Range([(0, tM-1, tM//coarsening_factor)]),
                 "cj" : dace.subsets.Range([(0, tN-1, tN//coarsening_factor)])},
        schedule=dace.dtypes.ScheduleType.SoftHier_Sequential
    )

    for name in ["A", "B", "C"]:
        if name == "A" or name == "B":
                if name == "A":
                    access_str = ", ".join([f"i + gi * {tM}:i + gi * {tM} + {tM}", "0:K"])
                elif name == "B":
                    access_str = ", ".join(["0:K", f"j + gj * {tN}:j + gj * {tN} + {tN}"])
                elif name == "C":
                    access_str = ", ".join([f"i + gj * {tM}:i + gj * {tM} + {tM}", f"j + gj * {tN}:j + gj * {tN} + {tN}"])
                state.add_edge(thread_group_map_entry, f"OUT_{name}", thread_coarsened_map_entry, f"IN_{name}", dace.memlet.Memlet(f"{name}[{access_str}]"))
                thread_group_map_entry.add_out_connector(f"OUT_{name}")
                thread_coarsened_map_entry.add_in_connector(f"IN_{name}")
        if name == "C":
            access_str = ", ".join([f"i + gj * {tM}:i + gj * {tM} + {tM}", f"j + gj * {tN}:j + gj * {tN} + {tN}"])
            state.add_edge(thread_coarsened_map_exit, f"OUT_{name}", thread_group_map_exit, f"IN_{name}", dace.memlet.Memlet(f"{name}[{access_str}]"))
            thread_group_map_exit.add_in_connector(f"IN_{name}")
            thread_coarsened_map_exit.add_out_connector(f"OUT_{name}")

    block_tiled_map_entry, block_tiled_map_exit = main_state.add_map(
        name="block_tiled",
        ndrange={"bK" : dace.subsets.Range([(0, K-1, tK//coarsening_factor)])},
        schedule=dace.dtypes.ScheduleType.SoftHier_Sequential
    )

    accumulator_an = state.add_access("accumulator")
    accumulator_an.setzero = True
    state.add_edge(thread_coarsened_map_entry, None, accumulator_an, None, dace.memlet.Memlet(None))
    access_str = ", ".join([f"0:{coarsening_factor}*{coarsening_factor}", f"0:{tM//coarsening_factor}", f"0:{tN//coarsening_factor}"])
    state.add_edge(accumulator_an, None, block_tiled_map_entry, f"IN_accumulator", dace.memlet.Memlet(f"accumulator[{access_str}]"))
    block_tiled_map_entry.add_in_connector("IN_accumulator")
    thread_group_map_entry


    for name in ["A", "B"]:
        if name == "A":
            access_str = ", ".join([f"i + gi * {tM} + ci * {tM//coarsening_factor}:i + gi * {tM} + ci * {tM//coarsening_factor} + {tM//coarsening_factor}", "0:K"])
        elif name == "B":
            access_str = ", ".join(["0:K", f"j + gj * {tN} + cj * {tN//coarsening_factor}:j + gj * {tN} + cj * {tN//coarsening_factor} + {tN//coarsening_factor}"])

        state.add_edge(thread_coarsened_map_entry, f"OUT_{name}", block_tiled_map_entry, f"IN_{name}", dace.memlet.Memlet(f"{name}[{access_str}]"))
        block_tiled_map_entry.add_in_connector(f"IN_{name}")
        thread_coarsened_map_entry.add_out_connector(f"OUT_{name}")


    # Load
    local_access_nodes = dict()
    for name, shape in [("A", (tM//coarsening_factor, tK//coarsening_factor)), ("B", (tK//coarsening_factor, tN//coarsening_factor))]:
        block_tiled_map_entry.add_out_connector(f"OUT_{name}")
        arrn, arr = sdfg.add_array(name=f"local_{name}", shape=shape, dtype=input_float, storage=local_storage, transient=True)
        arrs[arrn] = arr
        an = state.add_access(f"local_{name}")
        local_access_nodes[f"local_{name}"] = an
        if name == "A":
            access_str = ", ".join([f"i + gi * {tM} + ci * {tM//coarsening_factor}:i + gi * {tM} + ci * {tM//coarsening_factor} + {tM//coarsening_factor}", 
                                    f"bK:bK+{tK//coarsening_factor}"])
        elif name == "B":
            access_str = ", ".join([f"bK:bK+{tK//coarsening_factor}", 
                                    f"j + gj * {tN} + cj * {tN//coarsening_factor}:j + gj * {tN} + cj * {tN//coarsening_factor} + {tN//coarsening_factor}"])
        state.add_edge(block_tiled_map_entry, f"OUT_{name}", an, None, dace.memlet.Memlet(f"{name}[{access_str}]"))

    # Connect local_A + local_B -> matmul -> accumulator
    matmul_tasklet = state.add_tasklet(name="mmad_redmule", inputs={"_in_local_a", "_in_local_b", "_in_accumulator"}, outputs={"_out_accumulator"},
                                       code=mmad_tasklet_str, language=dace.dtypes.Language.CPP)

    #for name in ["local_A", "local_B", "accumulate"]:
    #    state.add_edge()

    for name, an in local_access_nodes.items():
        state.add_edge(an, None, matmul_tasklet, "_in_" + name.lower(), dace.memlet.Memlet(name))
    state.add_edge(block_tiled_map_entry, f"OUT_accumulator", matmul_tasklet, "_in_accumulator", dace.memlet.Memlet("accumulator"))
    # accumulator_an2 = state.add_access("accumulator")
    # state.add_edge(matmul_tasklet, f"_out_accumulator", accumulator_an2, None, dace.memlet.Memlet("accumulator"))
    # state.add_edge(accumulator_an2, None, block_tiled_map_exit, "IN_accumulator", dace.memlet.Memlet("accumulator"))
    access_str = ", ".join([f"0:{coarsening_factor}*{coarsening_factor}", f"0:{tM//coarsening_factor}", f"0:{tN//coarsening_factor}"])
    state.add_edge(matmul_tasklet, "_out_accumulator", block_tiled_map_exit, "IN_accumulator", dace.memlet.Memlet(f"accumulator[{access_str}]"))
    block_tiled_map_entry.add_in_connector("IN_accumulator")
    block_tiled_map_exit.add_in_connector("IN_accumulator")
    block_tiled_map_entry.add_out_connector("OUT_accumulator")
    block_tiled_map_exit.add_out_connector("OUT_accumulator")


    # assign_tasklet = state.add_tasklet(name="assign", inputs={"_in_accumulator"}, outputs={"_out_C"}, code="_out_C = _in_accumulator")
    # state.add_edge(block_tiled_map_exit, "OUT_C", assign_tasklet, "_in_accumulator", dace.memlet.Memlet("accumulator")) , "_in_C"

    # accumulator_an3 = state.add_access("accumulator")
    # state.add_edge(block_tiled_map_exit, f"OUT_accumulator", accumulator_an3, None, dace.memlet.Memlet("accumulator"))
    # state.add_edge(accumulator_an3, None, assign_tasklet, "_in_accumulator", dace.memlet.Memlet("accumulator"))
    # state.add_edge(block_tiled_map_exit, f"OUT_accumulator", assign_tasklet, "_in_accumulator", dace.memlet.Memlet())

    # c_an2 = state.add_access("C")
    accumulator_an3 = state.add_access("accumulator")
    
    # state.add_edge(assign_tasklet, "_out_C", c_an2, None, dace.memlet.Memlet(f"C[{access_str}]"))
    # thread_coarsened_map_entry.add_out_connector(f"OUT_C")
    # state.add_edge(c_an2, None, thread_coarsened_map_exit, "IN_C", dace.memlet.Memlet(f"C[{access_str}]"))
    # state.add_edge(assign_tasklet, "_out_C", thread_coarsened_map_exit, "IN_C", dace.memlet.Memlet(f"C[{access_str}]"))
    # state.add_edge(block_tiled_map_exit, f"OUT_accumulator", thread_coarsened_map_exit, "IN_C", dace.memlet.Memlet(f"C[{access_str}]"))
    access_str = ", ".join([f"0:{coarsening_factor}*{coarsening_factor}", f"0:{tM//coarsening_factor}", f"0:{tN//coarsening_factor}"])
    state.add_edge(block_tiled_map_exit, f"OUT_accumulator", accumulator_an3, None, dace.memlet.Memlet(f"accumulator[{access_str}]"))
    access_str = ", ".join([f"i + gi * {tM} + ci * {tM//coarsening_factor}:i + gi * {tM} + ci * {tM//coarsening_factor} + {tM//coarsening_factor}", 
                            f"j + gj * {tN} + cj * {tN//coarsening_factor}:j + gj * {tN} + cj * {tN//coarsening_factor} + {tN//coarsening_factor}"])
    state.add_edge(accumulator_an3, None, thread_coarsened_map_exit, "IN_C", dace.memlet.Memlet(f"C[{access_str}]"))
    thread_coarsened_map_exit.add_in_connector("IN_C")
    
    cannon_nsdfg_node = CannonTransformer.apply_to(sdfg, map_entry=block_tiled_map_entry, transient=local_access_nodes["local_A"], options={"npe": gM, "gi": gi, "gj": gj})
    SplitHBMLoad.apply_to(sdfg, map_entry=block_tiled_map_entry, schedule_nsdfg_node = cannon_nsdfg_node, options={"npe": gM, "gi": gi, "gj": gj})

    return sdfg



split_scheme_A = (2, 2)
split_scheme_B = (2, 2)
split_scheme_C = (2, 2)
placement_scheme_A = (0, 3, 1)
placement_scheme_B = (0, 3, 1)
placement_scheme_C = (0, 3, 1)
tM = 128
tN = 128
tK = 128
if __name__ == "__main__":

    sdfg = _my_gen_matmul_sdfg(hardware_matmul_mnk=(tM, tN, tK),
                            global_storage=dace.dtypes.StorageType.SoftHier_HBM,
                            local_storage=dace.dtypes.StorageType.SoftHier_TCDM,
                            device_schedule=dace.dtypes.ScheduleType.SoftHier_Device,
                            thread_group_schedule=dace.dtypes.ScheduleType.SoftHier_Cluster,
                            thread_group_dims=(4, 4),
                            hbm_split_scheme=[split_scheme_A, split_scheme_B, split_scheme_C],
                            hbm_placement_scheme=[placement_scheme_A, placement_scheme_B, placement_scheme_C],
                            is_hbm_interleaved=False,
                            input_float=dace.float16,
                            output_float=dace.float16,
                            coarsening_factor=1,
                            mmad_tasklet_str="flex_redmule_trigger(_in_local_a, _in_local_b, _in_accumulator, REDMULE_FP_16);")
    sdfg.save("my_gemm.sdfgz")
    sdfg.validate()

{'s_local_A', 's_local_B'}
{'s_local_A': 'local_A', 's_local_B': 'local_B'}
{'s_local_A': 'A', 's_local_B': 'B'}


In [3]:

from IPython.display import Code
Code(sdfg.generate_code()[1].clean_code, language='cpp')
# sdfg.compile()


Waring: No `gpu_block_size` property specified on map gemm_entry. 
RedMule Dims [[1, 128, 128], [1, 128, 128]]
Generating NestedSDFG using SoftHierCodeGen
Generating NestedSDFG using SoftHierCodeGen
Generating Tasklet Using CPU Codegen
Generating Tasklet Using CPU Codegen


/scratch/dace4softhier/dace_devel/dace/codegen/targets/soft_hier/soft_hier.py:2097: UserWarning: No `gpu_block_size` property specified on map "gemm_entry". Falling back to the configuration entry `compiler.cuda.default_block_size`: 32,1,1. You can either specify the block size to use with the gpu_block_size property, or by adding nested `GPU_ThreadBlock` maps, which map work to individual threads. For more information, see https://spcldace.readthedocs.io/en/latest/optimization/gpu.html
  warnings.warn(


// #include <cuda_runtime.h>
// #include <dace/dace.h>
#include <math.h>
#include "flex_runtime.h"
#include "flex_redmule.h"
#include "flex_printf.h"
#include "flex_cluster_arch.h"
#include "flex_dma_pattern.h"

typedef struct GEMM_state_t {
    int filler;
}GEMM_state_t;



int __dace_init_cuda(struct GEMM_state_t *__state, int K, int M, int N);
int __dace_exit_cuda(struct GEMM_state_t *__state);

void nested_main_2_0_0(uint32_t A, uint32_t B, uint32_t accumulator, uint32_t local_A, uint32_t local_B, uint32_t K, uint32_t N, uint32_t bK, uint32_t gi, uint32_t gj) {
    long long _c;

    //Framecode generating state init_0...
    {
        // Start of state init_sync
        //Framecode generating state init_sync...
        {

            ///////////////////

            if (flex_is_dm_core()) {
                flex_dma_async_wait_all();
            }
            flex_intra_cluster_sync();

            ///////////////////

        }
        // End of state init_sync

    }
    for (_c = 0; (_c < 4); _c = (_c + 1)) {
        //Framecode generating state start...
        {
            // Start of state compute
            //Framecode generating state compute...
            // accumulator = accumulator;
            // s_local_A = s_local_A;
            // local_A = local_A;
            // s_local_B = s_local_B;
            // local_B = local_B;
            if (flex_is_first_core())
            {
                uint32_t _in_local_a = local_A + (16384 * (((bK / 512) + (3 * (_c % 2))) % 4)) * 2;
                uint32_t _in_local_b = local_B + (16384 * (((bK / 512) + (3 * (_c % 2))) % 4)) * 2;
                uint32_t _in_accumulator = accumulator;

                ///////////////////
                flex_redmule_trigger(_in_local_a, _in_local_b, _in_accumulator, REDMULE_FP_16);
                flex_redmule_wait();
                ///////////////////

            }
            // accumulator = accumulator;
            // End of state compute

        }
        {
            // Start of state communication
            //Framecode generating state communication...
            // local_A = local_A;
            // copy_memory: local_A -> s_local_A
            // is_sync = False
            if (flex_is_dm_core())
            {
                bare_dma_start_1d(remote_xy(gi,((gj + 1) % 4),local_A+(((bK / 512) + (3 * ((_c + 1) % 2))) % 4) * 32768), local(local_A + (16384 * (((bK / 512) + (3 * (_c % 2))) % 4))*2), 32768);
                flex_dma_async_wait_all();
            }
            // s_local_A = s_local_A;
            // local_B = local_B;
            // copy_memory: local_B -> s_local_B
            // is_sync = False
            if (flex_is_dm_core())
            {
                bare_dma_start_1d(remote_xy(((gi + 1) % 4),gj,local_B+(((bK / 512) + (3 * ((_c + 1) % 2))) % 4) * 32768), local(local_B + (16384 * (((bK / 512) + (3 * (_c % 2))) % 4))*2), 32768);
                flex_dma_async_wait_all();
            }
            // s_local_B = s_local_B;
            // End of state communication

        }
        if ((bK < ((K - 1) - 512))) {
            //Framecode generating state hbm_split_load_start...
            if ((gj == ((4 - _c) - 1))) {
                {
                    // Start of state local_A_hbm
                    //Framecode generating state local_A_hbm...
                    // A = A;
                    // copy_memory: A -> s_local_A
                    // is_sync = False
                    if (flex_is_dm_core())
                    {
                        flex_dma_async_2d(remote_xy(gi,gj,local_A+(((bK / 512) + 1) % 4) * 32768), hbm_addr(A + ((bK + (128 * ((gi + gj) % 4))) + 512)*2), 128*2, 128*2, K*2, 128);
                        flex_dma_async_wait_all();
                    }
                    // s_local_A = s_local_A;
                    // End of state local_A_hbm

                }
            }
            if ((gi == ((4 - _c) - 1))) {
                {
                    // Start of state local

In [4]:
# sdfg

In [5]:
# sdfg.compile()
# import numpy as np
K = 4096
M = 512
N = 512
# A_host = np.random.rand(M, K).astype(np.float16)
# B_host = np.random.rand(K, N).astype(np.float16)
A_host = np.ones((M, K), dtype=np.float16)
B_host = np.ones((K, N), dtype=np.float16)
C_host = np.zeros((M, N), dtype=np.float16)

# for i in range(M):
#     for j in range(K):
#         if np.random.rand() < 0.9:
#             A_host[i, j] /= 4.0
# for i in range(K):
#     for j in range(N):
#         if np.random.rand() < 0.9:
#             B_host[i, j] /= 4.0

for i in range(M):
    for j in range(K):
        if np.random.rand() < 0.7:
            A_host[i, j] = 0.0
for i in range(K):
    for j in range(N):
        if np.random.rand() < 0.7:
            B_host[i, j] = 0.0

G = A_host@B_host

start_address = 0x00000000
A_address = 64 + start_address
B_address = 64 + A_host.nbytes + start_address
C_address = 64 + A_host.nbytes + B_host.nbytes + start_address
G_address = 64 + A_host.nbytes + B_host.nbytes + C_host.nbytes + start_address
# create a uint32 np array to store the addresses
args = np.array([A_address, B_address, C_address, K, M, N, G_address], dtype=np.uint32)
# args = make_preload_elf_hbm_interleaved("output.elf", [A_host, B_host, C_host], [split_scheme_A, split_scheme_B, split_scheme_C], [placement_scheme_A, placement_scheme_B, placement_scheme_C], [(tM, tK), (tK, tN), (tM, tN)], start_addresses=[])
make_preload_elf("./output.elf", [args, A_host, B_host, C_host, G])

# make_preload_elf("/usr/scratch/badile111/dace4softhier/gvsoc/output.elf", [args, A_host, B_host, C_host])
# print(A_host@B_host)
# sdfg(A=A_host, B=B_host, C=C_host, M=M, N=N, K=K)

In [6]:
result = G


In [7]:
# print(A_host)

print(result[0][0])


            

393.0
